In [1]:
import os
import difflib
import shutil
from glob import glob
import pandas as pd
import numpy as np

from utils import get_chord_progressions, calculate_melody_results, flatten_chord_progression
from weimar_db_utils import flatten_wdb_chord_progression, get_offset, create_note_info
from melody import Melody

source = 'Weimar DB'

input_folder = '../data/Complete Examples Melodies Auto/v1.2/Weimar DB'
filepaths = [y for x in os.walk(input_folder) for y in glob(os.path.join(x[0], '*.mid'))]
cps = get_chord_progressions('..')

In [2]:
filepaths[9:10]

['../data/Complete Examples Melodies Auto/v1.2/Weimar DB/Branford Marsalis - The Nearness Of You.mid']

In [3]:
performer = 'Branford Marsalis'
song_name = 'The Nearness of You'
filename = filepaths[9]

print(filename)

wdb_info = pd.read_csv('../data/sources/WeimarDB/weimar_db_melody_info_with_pitch.csv')
song_wdb_info = wdb_info[
    (wdb_info['performer'] == performer) &
    (wdb_info['title'] == song_name)
]

if song_wdb_info['melid'].nunique() > 1:
    print('Duplicate!!!!!', song_wdb_info['melid'].unique())
    melid_idx = input('Enter correct index ')
    melid = list(song_wdb_info['melid'].unique())[int(melid_idx)]
    
    song_wdb_info = song_wdb_info[song_wdb_info['melid'] == melid]
    
note_info = create_note_info(song_wdb_info)

../data/Complete Examples Melodies Auto/v1.2/Weimar DB/Branford Marsalis - The Nearness Of You.mid


In [4]:
song_wdb_info['division'].value_counts()

8.0     143
4.0     132
6.0      97
10.0     96
18.0     61
12.0     57
9.0      48
17.0     45
20.0     41
19.0     37
24.0     34
14.0     32
26.0     27
22.0     21
11.0     15
7.0      14
2.0      13
1.0      13
13.0     11
5.0      10
16.0      9
3.0       8
23.0      7
Name: division, dtype: int64

In [5]:
melody = Melody(filename, '1.2', '..')
melody.setup()
melody.note_info = create_note_info(song_wdb_info)
melody.set_song_structure(cps[melody.song_name])

melody.min_measure = melody.note_info.measure.min()
melody.max_measure = int(np.ceil((melody.note_info.measure + (
        (melody.note_info.offset + melody.note_info.quant_duration) / melody.final_ticks_per_beat /
        melody.time_signature[0])).max())) - 1

In [7]:
melody.align_key()

In [8]:
melody.key

<music21.key.Key of D major>

In [9]:
melody.chord_progression_key, melody.chord_progression_minor

('F', False)

In [10]:
melody.transpose_semitones

3

In [11]:
melody.transpose()

In [12]:
one_bar = song_wdb_info[song_wdb_info['bar'] == 1]
if len(one_bar['pitch'].dropna()) > 0:
    starting_note_idx = one_bar['pitch'].dropna().index[0]
    notes_up_to_start = len(song_wdb_info.loc[:starting_note_idx]['pitch'].dropna())
    starting_measure = int(melody.note_info.iloc[notes_up_to_start]['measure'])
    print(starting_measure)
    melody.starting_measure = starting_measure
else:
    print('No bar 1!')

2


In [13]:
melody.split_melody(True)

Repetition 1, 666 notes
Repetition 2, 128 notes


True

In [14]:
results = calculate_melody_results(melody)
hcs = [(k, res['harmonic_consistency_mean']) for k, res in results.items()]
for i, hc in hcs:
    print(i, hc)

0 0.5716966966966966
1 0.5390625


In [15]:
linear_wdb_cp = flatten_wdb_chord_progression(performer, song_name, '..')
linear_cp = flatten_chord_progression(cps[melody.song_name])
sm = difflib.SequenceMatcher(None, linear_cp, linear_wdb_cp[:len(linear_cp)])
s = sm.ratio()

print(s)
print('--------')
for p in zip(linear_cp, linear_wdb_cp[:len(linear_cp)]):
    print(p)

D-maj
------
['A1', 'A2', 'B1', 'A3', 'A1', 'A2']
------
A1
['Dj7', 'Dj7', 'Dj7', 'Dj7', 'A-7', 'A-7', 'Dsus7', 'Dsus7', 'Gj7', 'Gj7', 'Gj7', 'Gj7', 'Go7', 'Go7', 'Go7', 'Go7', 'F#-7', 'F#-7', 'B7', 'B7', 'E-7', 'E-7', 'A7', 'A7', 'F#-7', 'F#-7', 'B7', 'B7', 'E-7', 'E-7', 'A7', 'A7']
------
A2
['Dj7', 'Dj7', 'Dj7', 'Dj7', 'A-7', 'A-7', 'Dsus7', 'Dsus7', 'Gj7', 'Gj7', 'Gj7', 'Gj7', 'Go7', 'Go7', 'Go7', 'Go7', 'F#-7', 'F#-7', 'B7', 'B7', 'E-7', 'E-7', 'A7', 'A7', 'Dj7', 'Dj7', 'C7', 'C7', 'D6', 'D6', 'D6', 'D6']
------
B1
['E-7', 'E-7', 'E-7', 'E-7', 'A7', 'A7', 'A7', 'A7', 'Dj7', 'Dj7', 'Dj7', 'Dj7', 'A-7', 'A-7', 'D7', 'D7', 'Gj7', 'Gj7', 'Gj7', 'Gj7', 'F#m7b5', 'F#m7b5', 'B7', 'B7', 'E7', 'E7', 'E7', 'E7', 'E-7', 'E-7', 'A7', 'A7']
------
A3
['Dj7', 'Dj7', 'Dj7', 'Dj7', 'A-7', 'A-7', 'Dsus7', 'Dsus7', 'Gj7', 'Gj7', 'Gj7', 'Gj7', 'Go7', 'Go7', 'Go7', 'Go7', 'F#-7', 'F#-7', 'B7', 'B7', 'E-7', 'E-7', 'A7', 'A7', 'Dj7', 'Dj7', 'B-7', 'B-7', 'E-7', 'E-7', 'A7', 'A7']
------
0.0138888888888

In [16]:
gtg = input('Good to go?')

if gtg == 'good':
    print('Finalising')
    for i, res in results.items():
        folder_midi = f'../data/split_melody/v1.2/{source}/'
        folder_csv = f'../data/split_melody_data/v1.2/{source}/'

        src_midi = folder_midi + res['out_filename']
        dst_midi = src_midi.replace('split_melody/v1.2', 'finalised/midi')


        src_csv = folder_csv + res['out_filename'].replace('.mid', '.csv')
        dst_csv = src_csv.replace('split_melody_data/v1.2', 'finalised/csv')

        if not os.path.exists(os.path.dirname(dst_midi)):
            os.makedirs(os.path.dirname(dst_midi))

        if not os.path.exists(os.path.dirname(dst_csv)):
            os.makedirs(os.path.dirname(dst_csv))

        shutil.copyfile(src_midi, dst_midi)
        shutil.copyfile(src_csv, dst_csv)

Good to go?good
Finalising
